In [3]:
from rdkit import Chem
from gensim.models import word2vec
from mol2vec.features import mol2alt_sentence, MolSentence, DfVec, sentences2vec
import numpy as np
import pandas as pd

In [12]:
df = pd.read_csv('../GraphDTA/PubChem_compound_2166.csv', sep=',', header=0)
cid2smile = df.loc[:,['cid','isosmiles']]
cid2smile

,cid,isosmiles
0,15,CC12CCC(=O)CC1CCC3C2CCC4(C3CCC4O)C
1,119,C(CC(=O)O)CN
2,137,C(CC(=O)O)C(=O)CN
3,176,CC(=O)O
4,187,CC(=O)OCC[N+](C)(C)C
...,...,...
2161,131704316,CCOCCP(CCOCC)CCP(CCOCC)CCOCC.CCOCCP(CCOCC)CCP(...
2162,131704319,C(CN(CC(=O)O)CC(=O)O)N(CCN(CC(=O)[O-])CC(=O)[O...
2163,131704321,CC(C1C(=O)NC(CSSCC(C(=O)NC(C(=O)NC(C(=O)NC(C(=...
2164,131704328,C1(C(C(OC(C1OC2C(C(C(C(O2)C(=O)O)P)O)O)C(=O)O)...


In [ ]:
cid2mol = {}
for cid, smile in cid2smile.values:
    mol = Chem.MolFromSmiles(smile)
    cid2mol[cid] = mol
cid2mol